In [33]:
# Tutorial based on http://androidkt.com/text-classification-using-attention-mechanism-in-keras/
%matplotlib inline

import tensorflow as tf
from keras_preprocessing import sequence
from tensorflow import keras
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Concatenate


In [29]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()


vocab_size = 10000
 
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2
 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size, start_char=start_id,
                                                                        oov_char=oov_id, index_from=index_offset)
 
word2idx = tf.keras.datasets.imdb.get_word_index()
 
idx2word = {v + index_offset: k for k, v in word2idx.items()}
 
idx2word[pad_id] = '<PAD>'
idx2word[start_id] = '<START>'
idx2word[oov_id] = '<OOV>'
 
max_len = 200
rnn_cell_size = 128
 
x_train = sequence.pad_sequences(x_train,
                                 maxlen=max_len,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
x_test = sequence.pad_sequences(x_test, maxlen=max_len,
                                truncating='post',
                                padding='post',
                                value=pad_id)

In [8]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


In [4]:
sequence_input = Input(shape=(max_len,), dtype='int32')

embedded_sequences = keras.layers.Embedding(vocab_size, 128, input_length=max_len)(sequence_input)

In [10]:
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM
                                     (rnn_cell_size,
                                      dropout=0.3,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_activation='relu',
                                      recurrent_initializer='glorot_uniform'), name="bi_lstm_0")(embedded_sequences)

layers_lstm_birnn = tf.keras.layers.LSTM(rnn_cell_size, dropout=0.2, return_sequences=True, return_state=True,
                                         recurrent_activation='relu', recurrent_initializer='glorot_uniform')

lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional(layers_lstm_birnn)(lstm)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

attention = Attention(128)
context_vector, attention_weights = attention(lstm, state_h)

output = keras.layers.Dense(1, activation='sigmoid')(context_vector)

model = keras.Model(inputs=sequence_input, outputs=output)

# summarize layers
print(model.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 128)     1280000     input_1[0][0]                    
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectional)       [(None, 200, 256), ( 263168      embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) [(None, 200, 256), ( 394240      bi_lstm_0[0][0]                  
                                                                 bi_lstm_0[0][1]                  
          

In [12]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])
 
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        min_delta=0,
                                                        patience=1,
                                                        verbose=0, mode='auto')



In [13]:
history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=200,
                    validation_split=.3, verbose=1, callbacks=[early_stopping_callback])


Train on 17500 samples, validate on 7500 samples
Epoch 1/10


  200/17500 [..............................] - ETA: 11:38 - loss: 0.6932 - acc: 0.4750

  400/17500 [..............................] - ETA: 8:46 - loss: 0.6931 - acc: 0.4875 

  600/17500 [>.............................] - ETA: 7:44 - loss: 0.6931 - acc: 0.4983

  800/17500 [>.............................] - ETA: 7:11 - loss: 0.6931 - acc: 0.5112

...

16800/17500 [===========================>..] - ETA: 14s - loss: nan - acc: 0.5050

17000/17500 [============================>.] - ETA: 10s - loss: nan - acc: 0.5047

17200/17500 [============================>.] - ETA: 6s - loss: nan - acc: 0.5045 

17400/17500 [============================>.] - ETA: 2s - loss: nan - acc: 0.5052

17500/17500 [==============================] - 416s 24ms/step - loss: nan - acc: 0.5047 - val_loss: nan - val_acc: 0.5067


In [14]:
result = model.evaluate(x_test, y_test)
print(result)


   32/25000 [..............................] - ETA: 10:43

   64/25000 [..............................] - ETA: 8:09 

   96/25000 [..............................] - ETA: 7:28

  128/25000 [..............................] - ETA: 7:00

  160/25000 [..............................] - ETA: 6:47

...

24928/25000 [============================>.] - ETA: 1s

24960/25000 [============================>.] - ETA: 0s

24992/25000 [============================>.] - ETA: 0s

25000/25000 [==============================] - 372s 15ms/step


[nan, 0.5]


In [28]:
model.predict(x_test[1].reshape(1, -1))

array([[nan]], dtype=float32)